<a href="https://colab.research.google.com/github/Ruby-L-i-n/CSDS234_FinalProject/blob/main/Generate_playlist_user_input_CB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing

In [ ]:
!pip install spotipy
!pip install validators
!pip install textblob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 5.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import random
import json
import time
import validators
import tkinter as tk
import re
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Spotify API
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from google.colab import userdata
client_id = userdata.get('client_id')
client_secret = userdata.get('client_secret')

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [ ]:
def is_integer(user_input):
    try:
        # Convert the user input to an integer
        int_value = int(user_input)
        # Check if the integer value is valid
        return validators.integer(int_value)
    except ValueError:
        # ValueError will be raised if the conversion to int fails
        return False

#User interaction
Prompts for url, prints the recommended playlst

In [31]:
#getting url
while True:
  # Prompt the user for a URL
  url = input("Please enter a URL: ")

  # Validate the URL
  if validators.url(url) and url.find("https://open.spotify.com/playlist/")>-1:
    print("You entered a valid URL:", url)
    break  # Exit the loop if a valid URL is entered
  else:
    print("Invalid URL. Please enter a valid URL.")

#getting number of wanted tracks
while True:
  # Prompt the user for a URL
  num_tracks = input("Enter number of songs (max: 500): ")

  # Validate the URL
  if num_tracks.isdigit() and int(num_tracks) < 501:
    print("You entered a valid number:", num_tracks)
    break  # Exit the loop if a valid number is entered
  else:
    print("Invalid number. Please enter a valid number.")

print("analyzing data...")
track_df2, playlist_df = get_dataframe(url)
track_df2.to_csv('/content/drive/Shareddrives/CSDS 234 Final Project/spotify_user_input_dataset/track_t1.csv', index=False)
playlist_df.to_csv('/content/drive/Shareddrives/CSDS 234 Final Project/spotify_user_input_dataset/playlist_t1.csv', index=False)
print("analyzing complete.")

time.sleep(.5)



Please enter a URL: https://open.spotify.com/playlist/6OcWuyMNKus06gmrh4FEiX?si=ByA55XdUTlynw9gg0Q1PIA&pi=u-BB_cxSI_Rr6v
You entered a valid URL: https://open.spotify.com/playlist/6OcWuyMNKus06gmrh4FEiX?si=ByA55XdUTlynw9gg0Q1PIA&pi=u-BB_cxSI_Rr6v
Enter number of songs (max: 500): 500
You entered a valid number: 500
analyzing data...
analyzing complete.


# Generate recommended

In [32]:
print("generating playlist....")
track_df_1 = pd.read_csv("/content/drive/Shareddrives/CSDS 234 Final Project/spotify_user_input_dataset/tracks_info_0-34443.csv")
track_df2 = pd.read_csv('/content/drive/Shareddrives/CSDS 234 Final Project/spotify_user_input_dataset/track_t1.csv')

track_df = pd.concat([track_df_1,track_df2],ignore_index=True)
track_df.drop_duplicates(inplace = True)
track_df.reset_index(drop=True, inplace = True)

track_df = pd.concat([track_df_1,track_df2],ignore_index=True)
track_df.drop_duplicates(inplace = True)
track_df.reset_index(drop=True, inplace = True)

def getSubjectivity(text):
  return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
  return TextBlob(text).sentiment.polarity

def getAnalysis(score, task="polarity"):
  if task == "subjectivity":
    if score < 1/3:
      return "low"
    elif score > 1/3:
      return "high"
    else:
      return "medium"
  else:
    if score < 0:
      return 'Negative'
    elif score == 0:
      return 'Neutral'
    else:
      return 'Positive'

def sentiment_analysis(df, text_col):
  # Perform sentiment analysis on text
  df['subjectivity'] = df[text_col].apply(getSubjectivity).apply(lambda x: getAnalysis(x,"subjectivity"))
  df['polarity'] = df[text_col].apply(getPolarity).apply(getAnalysis)
  return df

sentiment_analysis(track_df, "track_name")

def ohe_process(df, column, new_name):
  tf_df = pd.get_dummies(df[column])
  feature_names = tf_df.columns
  tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
  tf_df.reset_index(drop = True, inplace = True)
  return tf_df

def subjectivity(df):
  subject_ohe = ohe_process(df, "subjectivity", "subjectivity") * 0.3
  return subject_ohe

def polarity(df):
  polarity_ohe = ohe_process(df, "polarity", "polarity") * 0.5
  return polarity_ohe

def genre_tfidf(df):
  tfidf = TfidfVectorizer()
  tfidf_track_matrix = tfidf.fit_transform(df['genres'])
  genre_df = pd.DataFrame(tfidf_track_matrix.toarray())
  genre_df.columns = ['genre' + '|' + i for i in tfidf.get_feature_names_out()]
  return genre_df

def popularity_normalization(df):
  popularity = df[['popularity']]
  scaler = MinMaxScaler()
  popularity_df = pd.DataFrame(scaler.fit_transform(popularity), columns = ['popularity'])
  return popularity_df

  # audio features
def audio_features(df):
  #one-hot encoding
  key_ohe = ohe_process(df, "key", "key") * 0.5
  mode_ohe = ohe_process(df, "mode", "mode") * 0.5
  #normalization of other audio_features
  floats_cols = df.dtypes[df.dtypes == 'float64'].index.values #selects the audio feature colns
  floats = df[floats_cols].reset_index(drop = True)
  scaler = MinMaxScaler()
  floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns) * 0.2
  #putting them altogether
  audio_feats_df = pd.concat([key_ohe, mode_ohe, floats_scaled], axis=1)
  return audio_feats_df

def generate_feature_set(df):
  popularity = popularity_normalization(df)
  genre = genre_tfidf(df)
  subjectivity_names = subjectivity(df)
  audio_features_df = audio_features(df)

  feature_set = pd.concat([genre, popularity, subjectivity_names, audio_features_df], axis = 1)
  return feature_set

track_df1 = track_df2

#feature set of all tracks
total_tracks_features = generate_feature_set(track_df)
total_tracks_features['track_uri'] = track_df['uri']
total_tracks_features.drop_duplicates(inplace = True)
total_tracks_features

# feature set summary vector for a playlist
def input_playlist_features_vector (pid):
  playlist_uris = list(track_df1['uri'])
  input_playlist_features_df = total_tracks_features[total_tracks_features['track_uri'].isin(playlist_uris)]
  input_playlist_features_df = input_playlist_features_df.drop('track_uri', axis = 1)
  feature_vector = input_playlist_features_df.sum(axis=0)
  return feature_vector

# input_playlist_features_vector(playlist_df.loc[0,'pid'])
def generate_nonplaylist_features(pid):
    playlist_uris = set(track_df1['uri'])

    # Filter non-playlist features
    nonplaylist_features = total_tracks_features[~total_tracks_features['track_uri'].isin(playlist_uris)]
    num_false_values = len(~total_tracks_features['track_uri'].isin(playlist_uris)) - sum(~total_tracks_features['track_uri'].isin(playlist_uris))
    nonplaylist_features = nonplaylist_features.drop('track_uri', axis = 1)
    nonplaylist_features.reset_index(drop = True, inplace = True)

    return nonplaylist_features

def recommend_playlist(input_pid):
  input_playlist_features = input_playlist_features_vector(input_pid)
  nonplaylist_features = generate_nonplaylist_features(input_pid)

  playlist_uris = list(track_df1['uri'])
  nonplaylist_df = track_df[~track_df['uri'].isin(playlist_uris)]
  nonplaylist_df.reset_index(drop = True, inplace = True)
  nonplaylist_df['similarity'] = cosine_similarity(nonplaylist_features.values, input_playlist_features.values.reshape(1, -1))[:,0]

  nonplaylist_df = nonplaylist_df.sort_values('similarity',ascending = False)
  return nonplaylist_df

recommended = recommend_playlist(playlist_df.loc[0,'pid'])

generating playlist....


<ipython-input-32-159476ba687f>:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonplaylist_df['similarity'] = cosine_similarity(nonplaylist_features.values, input_playlist_features.values.reshape(1, -1))[:,0]


#Continue

In [33]:
print("generation complete.")
display(recommended)
time.sleep(.5)


generation complete.


,track_name,artist_name,artist_uri,genres,popularity,danceability,energy,key,loudness,mode,...,type,id,uri,track_href,analysis_url,duration_ms,time_signature,subjectivity,polarity,similarity
1184,Coffee (F***ing),Miguel,spotify:artist:360IAlyVv4PCEVjgyMZrxK,['r&b'],77,0.550,0.661,1,-7.363,1,...,audio_features,60kqoWSXSbiDJaDxgi8GG1,spotify:track:60kqoWSXSbiDJaDxgi8GG1,https://api.spotify.com/v1/tracks/60kqoWSXSbiD...,https://api.spotify.com/v1/audio-analysis/60kq...,313907,4,low,Neutral,0.840720
24288,Shockandawe,Miguel,spotify:artist:360IAlyVv4PCEVjgyMZrxK,['r&b'],77,0.766,0.902,0,-7.067,1,...,audio_features,11SY8jJkg1CtUgHx37qUlm,spotify:track:11SY8jJkg1CtUgHx37qUlm,https://api.spotify.com/v1/tracks/11SY8jJkg1Ct...,https://api.spotify.com/v1/audio-analysis/11SY...,150730,4,low,Neutral,0.840522
6888,Use Me,Miguel,spotify:artist:360IAlyVv4PCEVjgyMZrxK,['r&b'],77,0.432,0.867,9,-6.809,1,...,audio_features,2dpZuh58EaAhfLxzAH7AxT,spotify:track:2dpZuh58EaAhfLxzAH7AxT,https://api.spotify.com/v1/tracks/2dpZuh58EaAh...,https://api.spotify.com/v1/audio-analysis/2dpZ...,280267,4,low,Neutral,0.839200
8171,All I Want Is You,Miguel,spotify:artist:360IAlyVv4PCEVjgyMZrxK,['r&b'],77,0.776,0.603,5,-5.696,1,...,audio_features,5VA4Ispp52EA1sOqzMz3Av,spotify:track:5VA4Ispp52EA1sOqzMz3Av,https://api.spotify.com/v1/tracks/5VA4Ispp52EA...,https://api.spotify.com/v1/audio-analysis/5VA4...,295920,4,low,Neutral,0.836536
27998,I Wish You Would,Taylor Swift,spotify:artist:06HL4z0CvFAxyc27GXpf02,['pop'],100,0.653,0.893,0,-5.966,1,...,audio_features,5gRYrtvyVyaCRvLt56OfuV,spotify:track:5gRYrtvyVyaCRvLt56OfuV,https://api.spotify.com/v1/tracks/5gRYrtvyVyaC...,https://api.spotify.com/v1/audio-analysis/5gRY...,207440,4,low,Neutral,0.834657
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29380,My Favorite Moment of the Bee 3/Second,Original Broadway Cast Recording,spotify:artist:43qpnCSCXefs3GzxOrWzdz,['broadway'],3,0.353,0.275,4,-9.888,0,...,audio_features,1MsxKjQ5moDDHVBGeHEUwo,spotify:track:1MsxKjQ5moDDHVBGeHEUwo,https://api.spotify.com/v1/tracks/1MsxKjQ5moDD...,https://api.spotify.com/v1/audio-analysis/1Msx...,395027,4,high,Positive,0.165281
12770,Try - Live In Concert,John Mayer Trio,spotify:artist:68xuC1qXRhCUvGd4SSM655,['modern blues'],0,0.405,0.911,7,-6.842,0,...,audio_features,4OZtDR79Lwz1DblkJffzf3,spotify:track:4OZtDR79Lwz1DblkJffzf3,https://api.spotify.com/v1/tracks/4OZtDR79Lwz1...,https://api.spotify.com/v1/audio-analysis/4OZt...,413293,4,high,Positive,0.165064
29467,Angry Inch,Hedwig & The Angry Inch,spotify:artist:6NxcE1vZdlUZCnuitqIuHS,['show tunes'],0,0.498,0.873,8,-5.389,0,...,audio_features,7b5SsKiVaGArlVWRCYP6KL,spotify:track:7b5SsKiVaGArlVWRCYP6KL,https://api.spotify.com/v1/tracks/7b5SsKiVaGAr...,https://api.spotify.com/v1/audio-analysis/7b5S...,183293,4,high,Negative,0.161130
29472,Exquisite Corpse,Hedwig & The Angry Inch,spotify:artist:6NxcE1vZdlUZCnuitqIuHS,['show tunes'],0,0.209,0.929,11,-5.168,0,...,audio_features,6NSWY89cVbiNMdrd2mkjiw,spotify:track:6NSWY89cVbiNMdrd2mkjiw,https://api.spotify.com/v1/tracks/6NSWY89cVbiN...,https://api.spotify.com/v1/audio-analysis/6NSW...,183600,4,high,Positive,0.159461


In [34]:

print("analyzing accuracy of first ", num_tracks, " tracks....")
pid = extract_playlist_id(url)
print(metrics(pid))


analyzing accuracy of first  500  tracks....
r-precision value: 0.016159090909091033
ndcg value: 0.9912984138592889


# Data Processing


In [35]:
def get_dataframe(url):
  playlist_data = get_playlist_data(url)
  playlist_df = pd.DataFrame(playlist_data)
  clean_df = to_clean_df(playlist_data)
  track_df2 = process_tracks(clean_df)

  return track_df2, playlist_df

In [36]:
# get_playlist_data
def get_playlist_data(p_url):
  flattened_playlist = []
  track_ids = url_to_track_id(p_url) #find the ids
  tracks = []
  pid = extract_playlist_id(p_url)
  playlist = sp.playlist(pid)

  unique_album_ids = set()
  for i in range(len(track_ids)):
    time.sleep(.5)
    track = get_track_data(track_ids[i], i, pid) #return track data
    album = track['album_name']
    if album not in unique_album_ids:
         unique_album_ids.add(album)
    tracks.append(track) #add track to end

  playlist = sp.playlist(pid)
  playlist_info = {
    'pid': playlist['id'],
    'name': playlist.get('name', None),
    'collaborative': playlist['collaborative'],
    'modified_at': int(datetime.strptime(playlist['tracks']['items'][0]['added_at'], "%Y-%m-%dT%H:%M:%SZ").timestamp()),
    'num_tracks': len(track_ids),
    'num_albums': len(unique_album_ids),
    'num_followers': playlist['followers']['total']
  }

  for track_info in tracks:
    flattened_playlist.append({**playlist_info,**track_info})

  return flattened_playlist

def url_to_track_id(spotify_playlist_url):
  pid = extract_playlist_id(spotify_playlist_url)
  return get_tracks_id(pid)

def extract_playlist_id(url):
  start_index = url.find("https://open.spotify.com/playlist/") + len("https://open.spotify.com/playlist/")
  end_index = url.find("?", start_index)
  pid = url[start_index:end_index]
  return pid

def get_tracks_id(pid):
  music_id_list = []
  playlist = sp.playlist(pid)
  for item in playlist['tracks']['items']:
    music_track = item['track']
    music_id_list.append(music_track['id'])
  return music_id_list

def get_track_data(track_id, pos, pid):
  meta = sp.track(track_id)
  track_details = {"playlist_pid" : pid,
                   "track_pos": pos,
                   "track_name": meta['name'],
                   "track_uri": track_id,
                   "artist_name": meta['album']['artists'][0]['name'],
                   "artist_uri": meta['album']['artists'][0]['uri'],
                   "album_name": meta['album']['name'],
                   "album_uri": meta['album']['uri'],
                   "duration_ms": meta['duration_ms']}

  return track_details


In [37]:
# Create a Pandas DataFrame and cleaning
def to_clean_df(playlist_data):
  df = pd.DataFrame(playlist_data)
  df_copy = df.copy()
  clean_df = df_copy.drop_duplicates(['track_uri'])
  clean_df.reset_index(drop=True, inplace = True)
  track_df = clean_df[['track_name', 'track_uri', 'artist_name', 'artist_uri', 'album_name', 'album_uri', 'duration_ms']]
  return track_df

In [38]:
#processing tracks
def process_tracks(track_df):
  track_list = list(track_df.track_uri)
  artists_list = list(track_df.artist_uri)

  # obtaining artist and track uris to be used with Spotify API
  tracks_uri = []
  for track in track_list:
    tracks_uri.append(track)

  artists_uri = []
  for artist in artists_list:
    uri = artist.split("artist:")
    artists_uri.append(uri[1])

  time.sleep(5)
  audio_feats, artist_infos = gen_feats(track_df, artists_uri, tracks_uri)
  time.sleep(2)
  # Dataframe with audio_features
  track_info_df = pd.DataFrame()
  for batch in audio_feats:
    df = pd.DataFrame(batch)
    track_info_df = pd.concat([track_info_df, df], ignore_index=True)

  # data frame with information about artist (genres, popularity, etc)
  genres_list = []
  popularity_list = []
  artist_names = []

  for batch in artist_infos:
    for artist_info in batch['artists']:
      artist_names.append(artist_info['name'])
      genres_list.append(artist_info['genres'])
      popularity_list.append(artist_info['popularity'])

  track_names = list(track_df['track_name'])[0:len(genres_list)]
  artist_uri_fin = list(track_df['artist_uri'])[0:len(genres_list)]


  artist_info_data = {'track_name': track_names, 'artist_name': artist_names, 'artist_uri': artist_uri_fin, 'genres': genres_list, 'popularity': popularity_list}

  artist_info_df = pd.DataFrame(artist_info_data)

  # Combining the two dataframes
  fin_track_df = pd.concat([artist_info_df, track_info_df], axis=1)
  fin_track_df

  return fin_track_df

In [39]:
#generate features: round 1
def gen_feats(track_df, artists_uri, tracks_uri):

  # taking 50000 tracks at a time, we gather information about audio features as well as genres and popularity of song artists.
  audio_feats = []
  artist_infos = []
  num_tracks = len(track_df)
  num_counts,remainder = divmod(num_tracks,5000)
  counts = np.zeros((num_counts,50))
  min = 0
  for i in range(counts.shape[0]):
      counts[i] = list(np.arange(min, min + 5000,100))
      min += 5000

  j = 0
  for j in range(counts.shape[0]):
    for count in counts[j,:]:
      count = int(count)
      track_list_sub = tracks_uri[count:count+100]
      audio_feat = sp.audio_features(tracks=track_list_sub)
      audio_feats.append(audio_feat)
      artist_list_sub1 = artists_uri[count:count+50]
      artist_list_sub2 = artists_uri[count+50:count+100]
      songs_info1 = sp.artists(artist_list_sub1)
      songs_info2 = sp.artists(artist_list_sub2)
      artist_infos.append(songs_info1)
      artist_infos.append(songs_info2)
    time.sleep(10)

  num,remainder_2 = divmod(remainder,100)
  counts_2 = list(np.arange(num_tracks-remainder, num_tracks-remainder_2,100))
  for count in counts_2:
      count = int(count)
      track_list_sub = tracks_uri[count:count+100]
      audio_feat = sp.audio_features(tracks=track_list_sub)
      audio_feats.append(audio_feat)
      artist_list_sub1 = artists_uri[count:count+50]
      artist_list_sub2 = artists_uri[count+50:count+100]
      songs_info1 = sp.artists(artist_list_sub1)
      songs_info2 = sp.artists(artist_list_sub2)
      artist_infos.append(songs_info1)
      artist_infos.append(songs_info2)
  time.sleep(10)

  track_list_sub = tracks_uri[num_tracks-remainder_2:num_tracks+1]
  audio_feat = sp.audio_features(tracks=track_list_sub)
  audio_feats.append(audio_feat)
  artist_list_sub1 = artists_uri[num_tracks-remainder_2:num_tracks+1]
  if len(artist_list_sub1) > 50:
    songs_info_1 = sp.artists(artist_list_sub1[:50])
    songs_info_2 = sp.artists(artist_list_sub1[50:])
    artist_infos.append(songs_info_1)
    artist_infos.append(songs_info_2)
  else:
    songs_info1 = sp.artists(artist_list_sub1)
    artist_infos.append(songs_info1)

  return audio_feats, artist_infos

# Accuracy Tests

In [40]:
def metrics(input_pid):
  rec_song_info = return_to_features(recommended,num_tracks)
  original = input_playlist_features_vector(input_pid) #n x 1
  round_original = original.apply(round_ht)
  rec_song_info_df = pd.DataFrame(rec_song_info)

  r_precision_list = []
  ndcg_list = []
  for index, song in rec_song_info_df.iterrows():
    r_precision_list.append(r_precision(song,round_original))
    ndcg_list.append(ndcg(song,round_original))
  r_precision_val =  sum(r_precision_list)/len(r_precision_list)
  ndcg_val =  sum(ndcg_list)/len(ndcg_list)
  result_string = 'r-precision value: ' + str(r_precision_val) + '\nndcg value: ' + str(ndcg_val)
  return result_string

def return_to_features(recommended,num):
  rec_song_info = []
  recommended = recommended.head(int(num)) #30 x 26
  samples = generate_nonplaylist_features(pid) #m x n
  for song_index in recommended.index:
    rec_song_info.append(samples.loc[song_index])
  return rec_song_info

def r_precision(prediction, truth):
    prediction_set = set(prediction)
    truth_set = set(truth)
    intersect = prediction_set.intersection(truth_set)
    return float(len(intersect)) / len(truth_set)

def ndcg(predictions,truth):
    predictions_list = list(predictions)
    truth_list = list(truth)

    # Produces an ordered vector of 1.0 and 0.0
    score = [float(element in truth_list) for element in predictions_list]
    dcg  = np.sum(score / np.log2(1 + np.arange(1, len(score) + 1)))

    ones = np.ones([1,len(truth_list)])
    idcg = np.sum(ones / np.log2(1 + np.arange(1, len(truth_list) + 1)))

    return (dcg / idcg)


def round_ht(num):
  return round(num,100)